# Сравнение различных моделей машинного обучения

В данном семинаре мы вспомним, что было в прошлом году

In [ ]:
### импорты и документация

# Обработка данных
import numpy as np  # https://numpy.org/doc/stable/
import pandas as pd  # https://pandas.pydata.org/docs/

# Визуализация
import matplotlib.pyplot as plt  # https://matplotlib.org/stable/users/index.html
import seaborn as sns  # https://seaborn.pydata.org/tutorial.html

# Машинное обучение - Scikit-learn
from sklearn.datasets import fetch_california_housing  # https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html
from sklearn.model_selection import train_test_split, cross_val_score  # https://scikit-learn.org/stable/modules/cross_validation.html

# Модели
from sklearn.linear_model import LinearRegression, Ridge  # https://scikit-learn.org/stable/modules/linear_model.html
from sklearn.tree import DecisionTreeRegressor  # https://scikit-learn.org/stable/modules/tree.html
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor  # https://scikit-learn.org/stable/modules/ensemble.html
from sklearn.svm import SVR  # https://scikit-learn.org/stable/modules/svm.html
from sklearn.neighbors import KNeighborsRegressor  # https://scikit-learn.org/stable/modules/neighbors.html

# Метрики
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # https://scikit-learn.org/stable/modules/model_evaluation.html

# Бустинги
import xgboost as xgb  # https://xgboost.readthedocs.io/en/stable/python/python_api.html
import lightgbm as lgb  # https://lightgbm.readthedocs.io/en/stable/Python-Intro.html
from catboost import CatBoostRegressor  # https://catboost.ai/en/docs/concepts/python-reference_catboostregressor

# Дополнительно
import warnings
import time
warnings.filterwarnings('ignore')

Рассмотрим **California Housing Dataset** - классический датасет для задач регрессии, содержащий информацию о жилье в Калифорнии по данным переписи населения США 1990 года. Датасет широко используется для обучения и тестирования алгоритмов машинного обучения.

## Основные характеристики

### Размерность
- **Количество примеров:** 20,640
- **Количество признаков:** 8
- **Целевая переменная:** 1 (медианная стоимость дома)
- **Тип задачи:** Регрессия

### Источник данных
- **Оригинальный источник:** Перепись населения США 1990 года
- **Автор датасета:** Pace, R. Kelley, and Ronald Barry
- **Публикация:** "Sparse Spatial Autoregressions" (1997), Statistics and Probability Letters

## Описание признаков (Features)

| Признак | Описание | Диапазон значений | Тип |
|---------|----------|-------------------|-----|
| **MedInc** | Медианный доход населения в блоке (в десятках тысяч долларов) | 0.5 - 15.0 | Continuous |
| **HouseAge** | Медианный возраст домов в блоке (в годах) | 1 - 52 | Continuous |
| **AveRooms** | Среднее количество комнат на домохозяйство | 0.8 - 141.9 | Continuous |
| **AveBedrms** | Среднее количество спален на домохозяйство | 0.3 - 34.0 | Continuous |
| **Population** | Население блока | 3 - 35,682 | Continuous |
| **AveOccup** | Среднее количество членов домохозяйства | 0.69 - 1,243.3 | Continuous |
| **Latitude** | Широта блока (в градусах) | 32.54 - 41.95 | Continuous |
| **Longitude** | Долгота блока (в градусах) | -124.35 - -114.31 | Continuous |

## Целевая переменная (Target)

**MedianHouseValue** - Медианная стоимость дома для домохозяйств в пределах блока (в сотнях тысяч долларов)
- **Диапазон:** 0.15 - 5.0 (т.е. от $15,000 до $500,000)
- **Важно:** Значения обрезаны сверху на уровне $500,000

In [ ]:
## Особенности датасета

### Географическое распределение

# Код для визуализации географического распределения
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
import pandas as pd

housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = housing.target

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X['Longitude'], X['Latitude'], 
                     c=y, cmap='viridis', 
                     alpha=0.5, s=1)
plt.colorbar(scatter, label='Median House Value ($100k)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Географическое распределение цен на жилье в Калифорнии')
plt.show()

In [ ]:
### Статистические характеристики

# Базовая статистика по признакам
print("Статистика по признакам:")
print("-" * 60)
for col in housing.feature_names:
    data = X[col]
    print(f"{col:12} | Mean: {data.mean():8.2f} | Std: {data.std():8.2f} | "
          f"Min: {data.min():8.2f} | Max: {data.max():8.2f}")

print(f"\nTarget:      | Mean: {y.mean():8.2f} | Std: {y.std():8.2f} | "
      f"Min: {y.min():8.2f} | Max: {y.max():8.2f}")


## Известные проблемы и ограничения

### 1. Обрезание целевой переменной
- Все дома стоимостью более $500,000 записаны как $500,000
- Это создает искусственный потолок в данных
- Около 965 примеров (4.7%) имеют максимальное значение

### 2. Выбросы в признаках
- **AveRooms** и **AveBedrms** содержат экстремальные выбросы
- **AveOccup** имеет несколько аномально высоких значений
- Рекомендуется предобработка или удаление выбросов

### 3. Географическая кластеризация
- Данные естественно кластеризуются по географическим регионам
- Важно учитывать при разделении на train/test

### 4. Устаревшие данные
- Данные 1990 года могут не отражать современные тенденции
- Инфляция и изменения рынка недвижимости

In [ ]:
## Подготовка данных - Best Practices

# Рекомендуемая предобработка
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
import numpy as np

def prepare_california_housing():
    # Загрузка данных
    housing = fetch_california_housing()
    X = pd.DataFrame(housing.data, columns=housing.feature_names)
    y = housing.target
    
    # Обработка выбросов (опционально)
    # Ограничение экстремальных значений
    X['AveRooms'] = X['AveRooms'].clip(upper=X['AveRooms'].quantile(0.99))
    X['AveBedrms'] = X['AveBedrms'].clip(upper=X['AveBedrms'].quantile(0.99))
    X['AveOccup'] = X['AveOccup'].clip(upper=X['AveOccup'].quantile(0.99))
    
    # Создание дополнительных признаков
    X['Rooms_per_Bedroom'] = ### ваш код  - количество комнат на 1 спальню
    X['Bedroom_ratio'] =  ### ваш код - отношение спален к комнатам
    X['Household_density'] =  ### ваш код - плотность семей - количество жильцов на 1 человек
    
    # Логарифмирование скошенных признаков
    X['log_Population'] = np.log1p(X['Population'])
    X['log_AveOccup'] = np.log1p(X['AveOccup'])
    
    return X, y

# Использование
X, y = prepare_california_housing()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

В качестве метрики будем использовать **MAE (Mean Absolute Error)** - Средняя Абсолютная Ошибка - это метрика для оценки качества регрессионных моделей, которая показывает среднее абсолютное отклонение предсказанных значений от реальных.

## Математическая формула

$$MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|$$

Где:
- $n$ - количество наблюдений
- $y_i$ - реальное значение
- $\hat{y}_i$ - предсказанное значение
- $|...|$ - абсолютное значение (модуль)


In [ ]:
## Пример расчета

import numpy as np
from sklearn.metrics import mean_absolute_error

# Реальные значения
y_true = np.array([100, 150, 200, 250, 300])

# Предсказанные значения
y_pred = np.array([110, 140, 190, 260, 280])

# Ручной расчет MAE
errors = np.abs(y_true - y_pred)
print(f"Ошибки: {errors}")  # [10, 10, 10, 10, 20]
mae_manual = np.mean(errors)
print(f"MAE (ручной расчет): {mae_manual}")  # 12.0

# Расчет с помощью sklearn
mae_sklearn = mean_absolute_error(y_true, y_pred)
print(f"MAE (sklearn): {mae_sklearn}")  # 12.0

### Простая интерпретация
- **MAE = 12** означает, что в среднем наша модель ошибается на 12 единиц
- Если предсказываем цены домов в тысячах долларов, то MAE = 12 означает среднюю ошибку в $12,000

### Задание

Обучите на выборке и протестируйте следующие модели:

- Linear Regression - базовая линейная регрессия
- Ridge Regression - регрессия с L2-регуляризацией
- Decision Tree Regressor - дерево решений
- Random Forest Regressor - случайный лес
- Support Vector Regression (SVR) - метод опорных векторов
- K-Nearest Neighbors Regressor - метод k-ближайших соседей
- Gradient Boosting Regressor (sklearn)
- XGBoost Regressor
- LightGBM Regressor
- CatBoost Regressor

Рассчитайте для каждой модели:

- MAE (Mean Absolute Error)
- Время обучения

Сделайте выводы, оформите их